In [1]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import re
import numpy as np
from sqlalchemy import create_engine, exc
load_dotenv()


def E_mrt_realtime_arrival():
    username = os.getenv("ANDY_USERNAME")
    password = os.getenv("ANDY_PASSWORD")
    url = "https://api.metro.taipei/metroapi/TrackInfo.asmx"  # 列車到站資訊
    headers = {
        "Content-type": "text/xml; charset=utf-8"
    }
    xmldata = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
    xmlns:xsd="http://www.w3.org/2001/XMLSchema"
    xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
    <soap:Body>
    <getTrackInfo xmlns="http://tempuri.org/">
    <userName>{username}</userName>
    <passWord>{password}</passWord>
    </getTrackInfo>
    </soap:Body>
    </soap:Envelope>
    """

    response = requests.post(url=url, headers=headers, data=xmldata)
    df = pd.DataFrame(json.loads(response.text.split("<?xml")[0]))
    print("E_mrt_realtime_arrival finished")
    return (df)


def T_mrt_realtime_arrival(df: pd.DataFrame):
    def arrive_time_transform(x):
        if x == "列車進站":
            return (0)
        elif x == "資料擷取中":
            return (np.nan)
        else:
            try:
                text_list = x.split(":")
                mins, seconds = int(text_list[0]), int(text_list[1])
                return (int(mins*60+seconds))
            except:
                return (np.nan)

    df["arrive_time"] = df["CountDown"].apply(arrive_time_transform)
    df = df.loc[:, ['StationName', 'DestinationName',
                    'arrive_time', 'NowDateTime']]
    df.rename(columns={
        "StationName": "mrt_station_name",
        "DestinationName": "mrt_destination_name",
        "NowDateTime": "update_time"
    }, inplace=True)
    df["mrt_station_name"] = df["mrt_station_name"].str.rstrip("站")
    df["mrt_destination_name"] = df["mrt_destination_name"].str.rstrip("站")
    filename = datetime.strftime(datetime.now(), "%Y-%m-%d_%H-%M-%S")
    # df.to_csv(f"./{filename}mrt_realtime_arrival.csv",
    #           encoding="utf-8-sig", index=False)
    # return ("OK")
    print("T_mrt_realtime_arrival finished")
    return (df)




In [2]:
E_df = E_mrt_realtime_arrival()
T_df = T_mrt_realtime_arrival(df=E_df)

E_mrt_realtime_arrival finished
T_mrt_realtime_arrival finished


In [4]:
from google.cloud import storage

In [31]:
from google.cloud import storage
from google.oauth2 import service_account

key_path = r"D:\data_engineer\TIR_group2\TIR101_Group2\group2-422703-7c03158d7c95.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
bucket_name = "test123456"
source_file_name = r"D:\data_engineer\TIR_group2\TIR101_Group2\dags\test.csv"
destination_blob_name = "testblob.csv"
client = storage.Client(credentials=credentials)
bucket = client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)

blob.upload_from_filename(source_file_name)

Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/test123456/o?uploadType=multipart: {
  "error": {
    "code": 403,
    "message": "The billing account for the owning project is disabled in state closed",
    "errors": [
      {
        "message": "The billing account for the owning project is disabled in state closed",
        "domain": "global",
        "reason": "accountDisabled",
        "locationType": "header",
        "location": "Authorization"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>)

In [12]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'group2-422703-7c03158d7c95.json'

In [10]:
df = pd.DataFrame({"col1":[1,2,3]})
df.to_csv("test.csv",index=False)

In [25]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """上傳檔案到指定的 bucket。"""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# 調用函數
upload_blob('my-buckettest0508', 'test.csv', 'test.csv')


DefaultCredentialsError: File group2-422703-7c03158d7c95.json was not found.

In [19]:
storage_client = storage.Client()

In [20]:
bucket_name = 'test0508'

In [21]:
bucket = storage_client.bucket(bucket_name)

In [22]:
bucket.storage_class = 'COLDLINE' # Archive | Nearline | Standard
bucket.location = 'US' # Taiwan
bucket = storage_client.create_bucket(bucket) # returns Bucket object

C:\Users\andy\AppData\Local\Temp\ipykernel_8184\1929443809.py:2: DeprecationWarning: Assignment to 'Bucket.location' is deprecated, as it is only valid before the bucket is created. Instead, pass the location to `Bucket.create`.
  bucket.location = 'US' # Taiwan


Conflict: 409 POST https://storage.googleapis.com/storage/v1/b?project=group2-422703&prettyPrint=false: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.